In [2]:
import requests
import pandas as pd
from io import StringIO

url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/retrievebulkdataset?&key=ZXX854D8DD7H33NN6KCSE8JAJ&taskId=400b6a3045553655668da82a1b91a1b2&zip=false"

try:
    response = requests.get(url)
    response.raise_for_status()  # Check for errors

    # Assuming the response contains CSV data, you can use pandas to read it
    csv_data = StringIO(response.text)
    df = pd.read_csv(csv_data)

    # Save the DataFrame to a CSV file on your computer
    df.to_csv('weather_data.csv', index=False)

    print("Data saved successfully.")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")

Data saved successfully.


In [3]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import pandas as pd

# Azure Storage Account details
account_name = 'journeygroup'
account_key = '2MVcmcztJuQBAB2QkioYpHDfWwQbYgMK4dCxpM59RKa2nUU4TtMs7eoDSymhiSbYdc9aBoFAoqhW+ASte7GP7g=='
container_name = 'journeydata'

# Function to upload CSV file to Azure Storage Blob
def upload_csv_to_blob(csv_path, blob_name):
    # Read CSV file content
    df = pd.read_csv(csv_path)
    csv_content = df.to_csv(index=False)

    # Create a connection to Azure Storage Blob
    blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)

    # Upload the CSV file to Azure Storage Blob
    blob_client.upload_blob(csv_content, overwrite=True)

    print(f"File uploaded to Azure Storage Blob: {blob_name}")

# Example: Upload CSV file
upload_csv_to_blob('C:/Users/khand/OneDrive/Desktop/STA 3000/weather_data.csv', 'journeydata/weather_data.csv')

File uploaded to Azure Storage Blob: journeydata/weather_data.csv


In [1]:
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import StringIO  # Add this import for string buffer

# Your Azure Storage account information
account_name = 'journeygroup'
account_key = '2MVcmcztJuQBAB2QkioYpHDfWwQbYgMK4dCxpM59RKa2nUU4TtMs7eoDSymhiSbYdc9aBoFAoqhW+ASte7GP7g=='
container_name = 'journeydata'

# Load date_info_table and weather_data
date_info_table = pd.read_csv('C:/Users/khand/OneDrive/Desktop/STA 3000/DateInfoTable.csv')
weather_data = pd.read_csv('C:/Users/khand/OneDrive/Desktop/STA 3000/weather_data.csv')

# Convert 'full_date' column in date_info_table to datetime
date_info_table['full_date'] = pd.to_datetime(date_info_table['full_date'], format='%Y/%m/%d')

# Merge each row in weather_data with the corresponding row in date_info_table
merged_data = pd.merge(date_info_table, weather_data, left_index=True, right_index=True, suffixes=('_date_info', '_weather'))

# Drop the 'datetime' column
merged_data = merged_data.drop('datetime', axis=1)

# Display the merged data
print(merged_data)

# Upload merged data to Azure Storage
blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)
container_client = blob_service_client.get_container_client(container_name)

# Define the file name in Azure Storage
file_name = 'DateInfoTable_merged.csv'

# Convert DataFrame to CSV and upload to Azure Storage
merged_data_csv = merged_data.to_csv(index=False)

# Use StringIO to convert the string to bytes
csv_as_bytes = bytes(merged_data_csv, encoding='utf-8')
blob_client = container_client.get_blob_client(file_name)
blob_client.upload_blob(csv_as_bytes, overwrite=True)

print(f'Merged data uploaded to Azure Storage as {file_name}')


    date_id  full_date  tempmax
0         0 2023-01-01     12.2
1         1 2023-01-02     13.4
2         2 2023-01-03     13.0
3         3 2023-01-04     18.7
4         4 2023-01-05      9.4
..      ...        ...      ...
85       85 2023-03-23     14.3
86       86 2023-03-24     11.1
87       87 2023-03-25     12.8
88       88 2023-03-30      9.2
89       89 2023-03-31     12.8

[90 rows x 3 columns]
Merged data uploaded to Azure Storage as DateInfoTable_merged.csv
